# Modelling Flow Stress Using an Artificial Neural Network (ANN)

In [ ]:
# only for Google Colab
!wget -N "https://raw.githubusercontent.com/Institute-of-Metal-Forming/notebooks-ilsenburg/refs/heads/main/requirements.txt"
!wget -N "https://raw.githubusercontent.com/Institute-of-Metal-Forming/notebooks-ilsenburg/refs/heads/main/flow-stress/bst.csv"
!pip install -r "requirements.txt"

Import of required packages.

In [ ]:
import pandas as pd  # for data loading
import numpy as np  # for vectorized computations
import scipy.optimize as opt  # for least squares fitting
import plotly.express as px  # for plotting
from sklearn.neural_network import MLPRegressor  # providing neural network logic

## Loading the Experimental Data

We have the flow stress data in a long format CSV file and load it via `pandas` into a data frame.

In [ ]:
data = pd.read_csv(
    "bst.csv",  # file name
    encoding="utf8",  # use unicode to be safe on Windows systems (default on UNIX)
    sep=",",  # columns separated by comma
)
data

As the data is in long format (columns for temperature, strain and strain rate; rows for every data point), it is very comfortable to plot it with `plotly`.
But first we sort by temperature, then by strain rate and last by strain.

In [ ]:
data.sort_values(
    by=["temp", "rate", "strain"],  # multi-level sorting columns
    ascending=True,  # order from small to large
    inplace=True,  # use the existing data frame, do not copy the data
)
data

Now we can plot the data in dependence of the test conditions.

In [ ]:
px.line(
    data,
    "strain",  # strain on the x-axis
    "stress",  # stress on the y-axis
    color="temp",  # distinguish temperatures by colors
    facet_col="rate",  # draw multiple plots for distinct strain rates
    line_group="file",  # distinguish lines by file name, avoid zick-zack curves
)

## Normalize the Data

Artificial neural networks are sensitive to input and output scaling. Therefore, it is recommended to scale the data to a fixed range.

First, we calculate the minimum and maximum values of the data for all columns. Then, we store their difference additonally in the data frame.

In [ ]:
scales = data.drop("file", axis="columns").aggregate(["max", "min"]).T
scales["range"] = scales["max"] - scales["min"]
scales

We define utitlity functions that do the scaling using the values computed above.

In [ ]:
def normalize(d):
    normalized = d.copy()
    normalized[scales.index] = (d[scales.index] - scales["min"]) / scales["range"]
    return normalized


def denormalize(d):
    denormalized = d.copy()
    denormalized[scales.index] = d[scales.index] * scales["range"] + scales["min"]
    return denormalized

The normalized experimental data is obtain by applying the `normalize()` function.

In [ ]:
normalized_data = normalize(data)
normalized_data

## Defining the Model

The `MLPRegressor` class provides the neural network for us. It has a variety of options to alter the network construction and fitting behavior. See the [docs](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor) for more information. Here, we only set the count and size (neurons per layer) of the hidden layers. The size of input and output layer is determined later when fitting.

In [ ]:
model = MLPRegressor(
    hidden_layer_sizes=[100, 100, 100],
)

## Fit the Model to the Data

The model object has a method `fit()` which takes the input and output data to fit on as multi-dimensional arrays.
The fit is directly stored within the object.

In [ ]:
model.fit(
    normalized_data[["strain", "temp", "rate"]].to_numpy(),
    normalized_data["stress"].to_numpy(),
)

## Plot the Model Predictions Counter the Data

First we create a fine cartesian raster to evaluate the model on, so we get smooth curves of the model prediction.

In [ ]:
strains = np.linspace(0, 1.2, 50)  # strain with 50 points between 0 and 1.2
temps = (
    data.temp.unique()
)  # take only the distinct temperatures that are present in the data
rates = data.rate.unique()  # respectively

grid = pd.MultiIndex.from_product(
    [temps, rates, strains], names=["temp", "rate", "strain"]
)
grid

The raster is an index for a data frame, but we want it as an actual dataframe for easier computation.

In [ ]:
model_data = pd.DataFrame(index=grid).reset_index()
model_data["stress"] = pd.NA
model_data

Again, we have to normalize this raster to be able to evaluate the fitted model on it.

In [ ]:
normalized_model_data = normalize(model_data)
normalized_model_data

Then we apply the fitted model with its `predict()` method and save the results in an additonal column in the data frame.

In [ ]:
normalized_model_data["stress"] = model.predict(
    normalized_model_data[["strain", "temp", "rate"]].to_numpy()
)
normalized_model_data

Now we can denormalize the model results to get the actual predictions.

In [ ]:
model_data = denormalize(normalized_model_data)
model_data

We can plot the results as before with the data.

In [ ]:
px.line(model_data, "strain", "stress", color="temp", facet_col="rate")

To plot both in comparison, we first have to merge the data frames. We distinguish model predictions and experimental data by an additonal column containg a marker label.

In [ ]:
combined_data = pd.concat(
    [model_data, data],  # list of the frames to combine
    keys=["model", "exp"],  # list of marker labels in same order as above
    names=["type", "index"],  # names of the marker column and the index column
)
combined_data.reset_index(
    level=0, inplace=True
)  # make the type column a normal column (was an index column)
combined_data.fillna(
    value={"file": ""}, inplace=True
)  # fill missing file in model results
combined_data

Now we plot the data as before, but distinguish the origin by lime style (solid and dashed).

In [ ]:
px.line(
    combined_data,
    "strain",
    "stress",
    color="temp",
    facet_col="rate",
    line_dash="type",
    line_group="file",
)